### Строим логистическую регрессию - угадываем пол спортсмена по признакам

https://www.kaggle.com/rio2016/olympic-games

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv( 'athletes.csv' )
data.head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11538 entries, 0 to 11537
Data columns (total 11 columns):
id             11538 non-null int64
name           11538 non-null object
nationality    11538 non-null object
sex            11538 non-null object
dob            11537 non-null object
height         11208 non-null float64
weight         10879 non-null float64
sport          11538 non-null object
gold           11538 non-null int64
silver         11538 non-null int64
bronze         11538 non-null int64
dtypes: float64(2), int64(4), object(5)
memory usage: 991.6+ KB


#### Попробуем угадать пол на основе роста, веса и вида спорта

Посмотрим много ли в наших признаках пустых значений

In [4]:
data[ pd.isnull( data['height'] ) ].head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
12,258556239,Abbas Qali,IOA,male,10/11/92,NaN,NaN,aquatics,0,0,0
47,469953606,Abdoullah Bamoussa,ITA,male,6/8/86,NaN,NaN,athletics,0,0,0
50,325809293,Abdul Omar,GHA,male,10/3/93,NaN,NaN,boxing,0,0,0
52,262868423,Abdulaziz Alshatti,IOA,male,10/30/90,NaN,NaN,fencing,0,0,0
56,897549624,Abdullah Hel Baki,BAN,male,8/1/89,NaN,NaN,shooting,0,0,0


In [5]:
print 'Для height пустых строк {}'.format( len( data[ pd.isnull( data['height'] ) ] ) )
print 'Для weight пустых строк {}'.format( len( data[ pd.isnull( data['weight'] ) ] ) )
print 'Для sport пустых строк {}'.format( len( data[ pd.isnull( data['sport'] ) ] ) )
print 'Всего строк в наборе {}'.format( len( data ) )

Для height пустых строк 330
Для weight пустых строк 659
Для sport пустых строк 0
Всего строк в наборе 11538


In [6]:
data['height'].unique()

array([ 1.72,  1.68,  1.98,  1.83,  1.81,  1.8 ,  2.05,  1.93,  1.65,
        1.7 ,  1.75,   nan,  1.61,  1.78,  1.76,  2.1 ,  1.73,  1.85,
        1.77,  1.9 ,  1.86,  1.74,  1.6 ,  2.07,  1.88,  1.66,  1.62,
        1.87,  2.03,  1.69,  1.82,  1.89,  1.94,  1.95,  1.71,  1.84,
        1.91,  1.67,  2.02,  1.58,  1.63,  1.79,  1.97,  1.56,  1.55,
        1.57,  1.46,  1.92,  1.64,  1.53,  1.99,  1.96,  2.  ,  2.04,
        1.47,  1.52,  2.01,  1.51,  1.59,  2.08,  1.37,  1.5 ,  1.45,
        2.06,  1.54,  2.11,  1.43,  1.49,  1.33,  1.48,  1.44,  2.13,
        2.09,  2.21,  2.18,  1.21,  1.38,  1.34,  2.15,  2.17,  1.42,
        1.4 ,  2.14])

In [7]:
# обнуляем ячейки с отсутствием данных
data = data[ pd.isnull( data['height'] ) == 0 ]
data = data[ pd.isnull( data['weight'] ) == 0 ]

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10858 entries, 0 to 11537
Data columns (total 11 columns):
id             10858 non-null int64
name           10858 non-null object
nationality    10858 non-null object
sex            10858 non-null object
dob            10858 non-null object
height         10858 non-null float64
weight         10858 non-null float64
sport          10858 non-null object
gold           10858 non-null int64
silver         10858 non-null int64
bronze         10858 non-null int64
dtypes: float64(2), int64(4), object(5)
memory usage: 1017.9+ KB


In [9]:
from sklearn.linear_model import LogisticRegression

In [11]:
# попробуем выбрать какие-нибудь признаки
selectedColumns = data[ [ 'height', 'sport', 'sex' ] ]

# столбец sport является категориальной переменой
# переведем ее в значения 0 и 1, добавив столбцы с соответствующими названиями
X = pd.get_dummies( selectedColumns, columns = [ 'sport' ] )

# столбец sex является целевой переменной, удаляем его из X
del X['sex']
X.head()

,height,sport_aquatics,sport_archery,sport_athletics,sport_badminton,sport_basketball,sport_canoe,sport_cycling,sport_equestrian,sport_fencing,...,sport_rugby sevens,sport_sailing,sport_shooting,sport_table tennis,sport_taekwondo,sport_tennis,sport_triathlon,sport_volleyball,sport_weightlifting,sport_wrestling
0,1.72,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.68,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1.98,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.83,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1.81,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# целевая переменная (столбец sex) снова является категориальной
# переведем значения столбца в числа, оставив один столбец

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [13]:
le.fit( data['sex'] )

LabelEncoder()

In [14]:
le.classes_

array(['female', 'male'], dtype=object)

In [15]:
# пример "расшировки" столбца sex

le.transform( [ 'male', 'female', 'male' ] )

array([1, 0, 1], dtype=int64)

In [16]:
# записываем в переменную y преобразованный столбец sex

y = pd.Series( data = le.transform( data['sex'] ) )
y.head()

0    1
1    0
2    1
3    1
4    1
dtype: int64

In [17]:
model = LogisticRegression()

In [18]:
# обучаем модель

model.fit( X, y )
predictions = model.predict_proba( X )

In [19]:
predictions[:5]

array([[ 0.58402666,  0.41597334],
       [ 0.78420659,  0.21579341],
       [ 0.03479216,  0.96520784],
       [ 0.35517459,  0.64482541],
       [ 0.18261018,  0.81738982]])

In [20]:
# сравниваем факт с предсказаниями
# (да, сравнивать предсказания с самими данными не круто, лучше было разбить на обучающую и тестовую выборку)
# это будет в следующем занятии

zip( predictions[:, 1], y )

[(0.41597334302050898, 1),
 (0.21579341292465296, 0),
 (0.96520783780453567, 1),
 (0.64482540606131955, 1),
 (0.81738981765886487, 1),
 (0.72623477901705291, 1),
 (0.90035185728820954, 1),
 (0.88348041321631243, 1),
 (0.68730050018040267, 0),
 (0.2099340417364523, 0),
 (0.34954517360353071, 1),
 (0.52079922722417693, 1),
 (0.13138703267307345, 0),
 (0.47825316745653046, 0),
 (0.54282022078640757, 0),
 (0.36566298849837947, 1),
 (0.58425190604290533, 1),
 (0.58425190604290533, 0),
 (0.94811392525008908, 1),
 (0.45054506611838785, 1),
 (0.81635022818107739, 1),
 (0.59024022578964241, 1),
 (0.7393032572317747, 1),
 (0.89989846573941534, 1),
 (0.52079922722417693, 1),
 (0.583103850334038, 1),
 (0.85180399463652234, 1),
 (0.86871875353665995, 1),
 (0.85180399463652234, 1),
 (0.62637296661529374, 1),
 (0.84694125497708317, 1),
 (0.8266378190088215, 1),
 (0.34954517360353071, 1),
 (0.59024022578964241, 1),
 (0.40998405554234535, 1),
 (0.8730292524581782, 1),
 (0.5575071929553912, 1),
 (0.3439

In [21]:
model.score(X, y)

0.79526616319764232